In [1]:
import random
import operator
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import networkx as nx

In [2]:
class Address:
    def __init__(self, id_):
        self.id_ = id_
    
    def distance(self, address):
        global distanceMatrix
        return distanceMatrix[self.id_][address.id_]

    def __repr__(self):
        return str(self.id_)

In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0.0
        self.fitness = 0.0
    
    def routeDistance(self):
        if(self.distance == 0):
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromAddress = self.route[i]
                toAddress = None
                if(i + 1 < len(self.route)):
                    toAddress = self.route[i + 1]
                else: 
                    toAddress = self.route[0]
                pathDistance += fromAddress.distance(toAddress)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if(self.fitness == 0):
            self.fitness = 1 / float(self.routeDistance())
        
        return self.fitness

In [5]:
def createRoute(addressList):
    route = random.sample(addressList, len(addressList))
    return route

def initialPopulation(populationSize, addressList):
    population = []
    for i in range(0, populationSize):
        population.append(createRoute(addressList))
    return population

def rankRoutes(population):
    fitnessResults = {}
    for i in range(0, len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key=operator.itemgetter(1), reverse=True)

def selection(populationRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(populationRanked), columns=["Index", "Fitness"])
    df["cumulative_sum"] = df.Fitness.cumsum()
    df["cumulative_perc"] = 100 * df.cumulative_sum / df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(populationRanked[i][0])
    for i in range(0, len(populationRanked) - eliteSize):
        pick = 100 * random.random()
        for i in range(0, len(populationRanked)):
            if(pick <= df.iat[i, 3]):
                selectionResults.append(populationRanked[i][0])
                break
    return selectionResults

def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent2))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0, eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool) - i - 1])
        children.append(child)
    return children

def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            address1 = individual[swapped]
            address2 = individual[swapWith]

            individual[swapped] = address2
            individual[swapWith] = address1

    return individual

def mutatePopulation(population, mutationRate):
    mutatedPopulation = []

    for individual in range(0, len(population)):
        mutatedIndividual = mutate(population[individual], mutationRate)
        mutatedPopulation.append(mutatedIndividual)
    return mutatedPopulation

def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    progress = []
    pop = initialPopulation(popSize, population)
    progress.append(1 / rankRoutes(pop)[0][1])
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))

    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])

    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    print(bestRoute)
    
    return bestRoute, 1 / rankRoutes(pop)[0][1]
#     plt.plot(progress)
#     plt.ylabel('Distance')
#     plt.xlabel('Generation')
#     plt.show()

In [ ]:
# distanceMatrix = [[0,       2,      np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
#                   [2,       0,      3,      np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
#                   [np.inf,  3,      0,      3     , np.inf, np.inf, 1     ,      2,      1],
#                   [np.inf,  np.inf, 3,      0     , 2     , 2     , np.inf, np.inf, np.inf],
#                   [np.inf,  np.inf, np.inf, 2     , 0     , 2     , np.inf, np.inf, np.inf],
#                   [np.inf,  np.inf, np.inf, 2     , 2     , 0     , np.inf, np.inf, np.inf],
#                   [np.inf,  np.inf, 1     , np.inf, np.inf, np.inf, 0,      np.inf, np.inf],
#                   [np.inf,  np.inf, 2     , np.inf, np.inf, np.inf, np.inf, 0,      np.inf],
#                   [np.inf,  np.inf, 1     , np.inf, np.inf, np.inf, np.inf, np.inf, 0     ]]
                  
# crossroadList = []


# geneticAlgorithm(population=crossroadList, popSize=100, eliteSize=20, mutationRate=0.01, generations=200)

In [44]:
# distanceMatrix = np.full((403, 403), np.inf)

crossroadList = []
addressList = []
with open('../data/graphs/temp/pungert-distances-06-19-2021-14-02-38.net', 'r', encoding="utf-8") as f:
    for line in f.readlines():
        if(line.strip().endswith("crossroad")):
            vals = line.strip().split(" ")
            crossroadList.append(Address(int(vals[0]) - 1))
        elif(line.strip().endswith("address")):
            vals = line.strip().split(" ")
            addressList.append(Address(int(vals[0]) - 1))

In [34]:
# graph = nx.read_pajek(f'../data/graphs/temp/tolmin-distances-06-19-2021-14-08-18.net')
graph = nx.read_pajek(f'../data/graphs/temp/pungert-distances-06-19-2021-14-02-38.net')

distances = dict(nx.all_pairs_dijkstra_path_length(graph, weight=lambda u, v, d: int(d[0]['distance'])))

crossroadList = list(filter(lambda x: x[1] == 'crossroad', (map(lambda x: (x[0], graph.nodes[x[1]]['node_type']), enumerate(graph.nodes)))))

In [28]:
distanceMatrix = np.full((len(graph.nodes), len(graph.nodes)), 0)
for ix_1, node_1 in enumerate(graph.nodes()):
    for ix_2, node_2 in enumerate(graph.nodes()):
        distanceMatrix[ix_1][ix_2] = distances[node_1][node_2]

In [43]:
type(crossroadList[1])

__main__.Address

In [46]:
bestRoute, finalDistance = geneticAlgorithm(population=crossroadList, popSize=100, eliteSize=20, mutationRate=0.01, generations=200)

Initial distance: 2358.0
Final distance: 2358.0
[41, 38, 43, 40, 39, 42]
